In [ ]:
import pandas as pd
import os
from tqdm import tqdm
in_folder = None
out_folder = None
# files are csv in format sentence,verb,direct_object,preposition,prepositional_object,direct_object_i,preposition_i,object_i
# read in all files into one dataframe
# make sure to check that they are files and not folders
df = pd.DataFrame()
for file in tqdm(os.listdir(in_folder)):
    if os.path.isfile(os.path.join(in_folder,file)):
        # custom read in function
        # the headers are in the first line
        # the separator is a comma
        # there could be a comma in the first column, but the sixth column is always an integer
        with open(os.path.join(in_folder,file)) as f:
            header = f.readline().strip().split(",")
            part_list = []
            for line in f:
                if line.strip() == "":
                    continue
                parts = line.strip().split(",")
                while len(parts) != len(header):
                    # merge the first two columns until the length is correct
                    parts = [parts[0] + "," + parts[1]] + parts[2:]
                part_list.append(parts)
            df = pd.concat([df, pd.DataFrame(part_list,columns=header)])



In [ ]:
df

In [ ]:
# drop duplicates 
df.drop_duplicates(inplace=True)

# drop verbs with non-alphabetical characters
df = df[df.verb.str.isalpha()]


In [ ]:
# group dataframe by verb
grouped = df.groupby("verb")

In [ ]:
# create a dataframe for each verb
for verb, group in tqdm(grouped):
    group.to_csv(os.path.join(out_folder,verb+".csv"), index=False)
    # grouped_combination = group.groupby(["direct_object", "preposition"])

In [ ]:
import pandas as pd
# pd.set_option('display.max_rows', None)
out_folder = None

# Load the file for a certain verb
verb = "sneeze"
combo_i = 5
df = pd.read_csv(out_folder + verb + ".tsv", sep="\t")
# df = df[(df['direct_object'] != 'way') & (df['direct_object'] != 'shit') & (df['direct_object'] != 'fuck') & (df['direct_object'] != 'crap')]
pronouns_to_replace = ["he", "you", "it", "she", "I", "we", "yourself", "herself", "himself", "myself"]
# Replace pronouns in the 'direct_object' and 'prepositional_object' columns with 'PRON'
df['direct_object'] = df['direct_object'].replace(pronouns_to_replace, 'PRON')
df['prepositional_object'] = df['prepositional_object'].replace(pronouns_to_replace, 'PRON')

# print(df)

grouped_df = df.groupby('preposition').size().reset_index(name='count')

print(grouped_df)


sorted_df = grouped_df.sort_values(by='count', ascending=False)

#    print(f"Group: {group['direct_object']} {group['preposition']} {group['prepositional_object']} ({group['count']})")
# Iterate through groups and print content
for i, (index, group) in enumerate(sorted_df.iterrows()):
    print(f"{group['preposition']}({group['count']})\n")
    subgroup = df[(df['preposition'] == group['preposition'])]
    
    
    
    further_subgroup = subgroup.groupby(['direct_object', 'preposition', 'prepositional_object']).size().reset_index(name='count')
    
    # Sort the DataFrame by 'count' column in descending order
    further_subgroup_sorted = further_subgroup.sort_values(by='count', ascending=False)

    # Further group the subgroup by 'preposition' and print the first 5 items of each subgroup
    for item_num, item in further_subgroup_sorted.head(combo_i).iterrows():
        combo_count = further_subgroup[(further_subgroup['direct_object'] == item['direct_object']) & (further_subgroup['preposition'] == item['preposition']) & (further_subgroup['prepositional_object'] == item['prepositional_object'])]['count'].values[0]
        full_item = df[(df['direct_object'] == item['direct_object']) & (df['preposition'] == item['preposition']) & (df['prepositional_object'] == item['prepositional_object'])]
        print(combo_count, ' '.join(full_item[['direct_object', 'preposition', 'prepositional_object']].values.tolist()[0]))
        print("\t", end="")
        print("\n\t".join(full_item.head(5)['sentence'].values.tolist()))
        print("\n")
        
    print("---------------------------------------------------")
    
    #     # Break after the first 50 groups
    # if i == 50:
    #     break
